# Student Performance Classifier 🎓

This notebook implements a Logistic Regression model to predict whether a student will **Pass (1)** or **Fail (0)** based on academic and lifestyle features.

## Phase 1: Setup & Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc
import os

# Configuration
DATA_PATH = '../dataset/student_data.csv'
SUBMISSION_DIR = '../submission'

os.makedirs(SUBMISSION_DIR, exist_ok=True)

# Load Data
df = pd.read_csv(DATA_PATH)
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

## Phase 2: Exploration & Analysis

In [ ]:
# Histograms for features
df.hist(figsize=(10, 8))
plt.tight_layout()
plt.show()

# Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

# Study Hours vs Result
plt.figure(figsize=(8, 6))
sns.boxplot(x='result', y='study_hours', data=df)
plt.title('Study Hours vs Pass/Fail')
plt.show()

## Phase 3: Building the Model

In [ ]:
# Prepare features and target
X = df.drop('result', axis=1)
y = df['result']

# Split: 80% Training, 20% Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Logistic Regression
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)
print("Model trained successfully.")

## Phase 4: Internal Evaluation

In [ ]:
# Predictions
y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Calculate Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f} (Required: >= 0.85)")
print(f"F1 Score: {f1:.4f} (Required: >= 0.80)")

if accuracy >= 0.85 and f1 >= 0.80:
    print("✅ All performance requirements met!")
else:
    print("❌ Performance requirements NOT met.")

## Phase 5: Validation & Submission

In [ ]:
print("Generating submission artifacts...")

# 1. Model Performance Report
report = classification_report(y_test, y_pred)
with open(os.path.join(SUBMISSION_DIR, 'model_performance.txt'), 'w') as f:
    f.write("Model Performance Metrics
")
    f.write("=========================

")
    f.write(f"Accuracy: {accuracy:.4f}
")
    f.write(f"F1 Score: {f1:.4f}

")
    f.write("Classification Report:
")
    f.write(report)

# 2. Confusion Matrix Plot
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(SUBMISSION_DIR, 'confusion_matrix.png'))
plt.show()

# 3. ROC Curve Plot
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.savefig(os.path.join(SUBMISSION_DIR, 'roc_curve.png'))
plt.show()

print(f"Submission files saved in: {os.path.abspath(SUBMISSION_DIR)}")